In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plotField2(ax, psi, Lx, Ly):
    x = np.linspace(0, Lx, psi.shape[0])
    y = np.linspace(0, Ly, psi.shape[1])
    X, Y = np.meshgrid(x, y)
    levels = np.linspace(-2.5, 2.5, 10)
    #ax.set_cmap('bwr')
    contour = ax.contourf(X, Y, np.transpose(psi), levels=levels)
    plt.colorbar(contour, ax=ax, label='Value